# ChatBot

> Konstantinos Mpouros <br>
> Github: https://github.com/konstantinosmpouros <br>
> Year: 2025 <br>

This project focuses on building an intelligent chatbot capable of providing accurate and context-aware responses. Designed to simulate human-like conversations, the chatbot is versatile enough for applications such as customer support, personal assistants, or educational tools.  

The project leverages state-of-the-art large language models (LLMs), offering the flexibility to choose between an open-source model from Hugging Face or GPT-4o. This dual approach ensures adaptability to different requirements, balancing performance, cost, and customization potential.  

## 1. Libraries

In [1]:
import os
from dotenv import load_dotenv
from huggingface_hub import login
import gc

# Chatbot UI
import gradio as gr

# Chatbot implementation
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from transformers import BitsAndBytesConfig

# RAG
from langchain_community.document_loaders import PyPDFLoader
from langchain.schema import Document
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

2025-01-21 13:04:25.448798: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-21 13:04:25.448856: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-21 13:04:25.451726: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-21 13:04:25.464127: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-21 13:04:26.2

## 2. Chatbot Implementation

* Load enviroment variables

In [2]:
load_dotenv()
os.environ['HUGGINGFACE_TOKEN'] = os.getenv('HUGGINGFACE_TOKEN')
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

* Login to hugging face hub

In [3]:
login(token=os.getenv('HUGGINGFACE_TOKEN'))

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /home/kostasbouros/.cache/huggingface/token
Login successful


* Define initial prompt

In [4]:
def init_prompts():
    system_message = """
        You are a helpfull ai assistant for an airline company named Aegean. Your tasks are the following:
            1. Provide customer service support to the user about Aegean policies, flights, services etc.
            2. Answer only in english, brief and clear, understanding first what the user needs.
            3. Be always polity and kind with any user! Always remember that you are made for customer support.
            4. If the user asks you to answer about something non related to aegean company and the relevant customer support, answer kindly that you cant answer that.
            5. If the user want you to execute a function call, answer only the JSON format and nothing more!! Nothing more!!
        When starting the conversation, greet kindly the user and then proceed to the customer support.
    """

    messages = [
        {"role": "system", "content": system_message},
    ]
    return messages

* Define a method to add in the history the user prompt and the response of the model

In [5]:
def add_to_history(history, role, prompt):
    history.append({'role': role, 'content': prompt})
    return history

* Define a function to remind in the model not to respond for other stuff exept those in the system message

In [6]:
def add_remider():
    prompt = """
        Remember, you are an ai assistant of the Aegean company providing customer support and answer only question that has to do with the company's info.
        if the user want to know something different answer kindly that you cant help with topic not relevand to aegean.
        Answer only in english, brief and clear. 
        If in order to answer to the user you need to call a function then respond only the JSON needed and nothing else!!
        Never reposnd the name of the function you call or here is the JSON format.
    """
    return prompt

* Initialize the model

In [7]:
def load_model(model_name):
    try:
        # Load tokenizer
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        tokenizer.pad_token = tokenizer.eos_token
        
        # Set quantization method
        quant_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_compute_dtype=torch.bfloat16,
            bnb_4bit_quant_type="nf4"
        )
        
        # Load model
        model = AutoModelForCausalLM.from_pretrained(model_name,
                                                     device_map="cuda",
                                                     quantization_config=quant_config)

        return tokenizer, model

    except Exception as ex:
        print(ex.args)
        gc.collect()
        torch.cuda.empty_cache()
        return None, None       

* Define a method to chat with the model

In [8]:
def llm_response(tokenizer, model, history, prompt):
    # Set up the message to be tokenized
    add_to_history(history, role='user', prompt=prompt)
    add_to_history(history, role='system', prompt=add_remider())
    tokenized_message = tokenizer.apply_chat_template(history, return_tensors="pt").to('cuda')

    # Generate response
    response = model.generate(tokenized_message, temperature=0.85, max_new_tokens=10000)
    generated_tokens = response[0][len(tokenized_message[0]):]

    # Decode response
    output = tokenizer.decode(generated_tokens , skip_special_tokens=True)
    if "assistant" in output:
        output = output.split("assistant", 1)[-1].strip()
    history = add_to_history(history, role='assistant', prompt=output)

    return output, history

In [9]:
def chat(tokenizer, model, history, prompt):
    llm_response(tokenizer, model, history, prompt)
    print(history[-1]['content'])

* Lets chat with the model

In [10]:
history = init_prompts()

In [11]:
tokenizer, model = load_model('meta-llama/Meta-Llama-3.1-8B-Instruct')

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [12]:
chat(tokenizer, model, history, 'Hi!!, l am konstantinos, you?')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)
2024-12-25 14:33:29.845039: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-25 14:33:29.913850: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN 

Hello Konstantinos! I'm an assistant for Aegean Airlines. How can I help you today?


In [13]:
chat(tokenizer, model, history, 'l would like to know where the name aegean come from!!')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


The name "Aegean" originates from the Aegean Sea, which is a significant body of water in Greece, surrounding many islands where the airline operates. This geographical connection reflects the airline's Greek roots and its primary service area.


In [14]:
chat(tokenizer, model, history, 'l would like to know if it a good company')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


As a reliable and reputable airline, Aegean has received numerous awards and accolades, including "Best Regional Airline in Europe" from the Air Transport Awards. We strive to provide excellent service and safe travels to our passengers. Would you like to know more about our services or routes?


In [15]:
chat(tokenizer, model, history, 'Ok thats some nice info!')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


I'm glad you found that information helpful, Konstantinos. If you're ready to move forward, what else would you like to know about Aegean Airlines? Our fleet, destinations, or something else?


In [16]:
chat(tokenizer, model, history, 'l would like to ask you about football actually, can you assist?')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


I'd be happy to help with Aegean-related topics, Konstantinos, but unfortunately, I'm not able to assist with football-related questions as it's not related to Aegean Airlines. If you'd like to know something about our flight routes, services, or more, I'd be happy to help.


In [17]:
chat(tokenizer, model, history, 'tell me about panathinaikos')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Panathinaikos is a well-known Greek sports club, but unfortunately, it's not directly related to Aegean Airlines. I'd be happy to help with a different question about Aegean Airlines, though. Perhaps you'd like to know about our partnership with Panathinaikos or any other Aegean-related topic?


In [18]:
chat(tokenizer, model, history, 'l dont want that, l want to know about the football only pls')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


I'd be happy to clarify, but unfortunately, I still can't assist with information about Panathinaikos' football team. My expertise lies within Aegean Airlines, and I'd be happy to provide information on our routes, services, or more. If you'd like to know something about Aegean's partnership with a sports club, I could try to provide that information.


In [19]:
chat(tokenizer, model, history, 'l want to know more about panathinaikos not aegean pls')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


I'm afraid I'm not able to provide information about Panathinaikos, as it's not related to Aegean Airlines. My purpose is to assist with questions about Aegean Airlines, and I'd be happy to help with that. If you'd like to know something about our services, routes, or more, I'd be happy to help.


In [20]:
chat(tokenizer, model, history, 'ok but do you remember my name?')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Yes, Konstantinos. I remember your name. How can I help you with an Aegean Airlines-related question now?


## 3. UI Chatbot

In [57]:
# Define your functions
def gradio_chat(user_input, history):
    if not tokenizer or not model:
        return "Model not loaded. Please check the configuration.", history

    response, history = llm_response(tokenizer, model, history, user_input)
    chat_history = [
        (entry["content"], None) if entry["role"] == "user" else (None, entry["content"]) 
        for entry in history if entry["role"] != "system"
    ]
    return chat_history, history

with gr.Blocks() as chat_interface:
    gr.Markdown("### Aegean AI Chat Assistant", elem_id="title")
    
    chatbot = gr.Chatbot()
    user_input = gr.Textbox(label="Your Message", placeholder="Type your message here...", lines=1)
    send_button = gr.Button("Send")
    
    # Logic for handling user input
    def handle_chat(input_text, history):
        chat_output, updated_history = gradio_chat(input_text, history)
        return chat_output, updated_history, ""

    # Bind send button and text box to chat logic
    send_button.click(handle_chat, inputs=[user_input, gr.State(history)], outputs=[chatbot, gr.State(history), user_input])
    user_input.submit(handle_chat, inputs=[user_input, gr.State(history)], outputs=[chatbot, gr.State(history), user_input])

# Load the model and initialize the chat history
history = init_prompts()
tokenizer, model = load_model('meta-llama/Meta-Llama-3.1-8B-Instruct')

# Launch the Gradio interface
chat_interface.launch()

/home/kostasbouros/.local/lib/python3.10/site-packages/gradio/components/chatbot.py:242: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

* Running on local URL:  http://127.0.0.1:7875

To create a public link, set `share=True` in `launch()`.


## 4. Streaming Chatbot

In [45]:
def llm_response_stream(tokenizer, model, history, prompt):
    # Set up the message to be tokenized
    add_to_history(history, role='user', prompt=prompt)
    add_to_history(history, role='system', prompt=add_remider())
    tokenized_message = tokenizer.apply_chat_template(history, return_tensors="pt").to('cuda')

    # Generate the full response
    response = model.generate(
        tokenized_message,
        temperature=0.85,
        max_new_tokens=10000
    )
    generated_tokens = response[0][len(tokenized_message[0]):]

    # Decode response and stream it
    output = ""
    for i, token_id in enumerate(generated_tokens):
        if i > 3:
            token_text = tokenizer.decode(token_id, skip_special_tokens=True)
            output += token_text
            yield token_text  # Stream each token back

    # Once streaming is complete, add the full output to history
    if "assistant" in output:
        output = output.split("assistant", 1)[-1].strip()
    history = add_to_history(history, role='assistant', prompt=output)

    # Return the final history (if needed)
    return history

In [51]:
tokenizer, model = load_model('meta-llama/Meta-Llama-3.1-8B-Instruct')

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [52]:
history = init_prompts()

In [53]:
prompt = 'hello!!'

response = []
for token in llm_response_stream(tokenizer, model, history, prompt):
    response.append(token)
    print(token)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Hello
!
 Welcome
 to
 Ae
ge
an
 Airlines
.
 How
 can
 I
 assist
 you
 today
?



In [54]:
prompt = 'tell me now about aegean!!'

response = []
for token in llm_response_stream(tokenizer, model, history, prompt):
    response.append(token)
    print(token)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


['I',
 "'d",
 ' be',
 ' happy',
 ' to',
 ' tell',
 ' you',
 ' about',
 ' Ae',
 'ge',
 'an',
 ' Airlines',
 '!\n\n',
 'A',
 'e',
 'ge',
 'an',
 ' Airlines',
 ' is',
 ' a',
 ' Greek',
 ' airline',
 ' that',
 ' operates',
 ' a',
 ' fleet',
 ' of',
 ' modern',
 ' aircraft',
 ',',
 ' offering',
 ' scheduled',
 ' and',
 ' charter',
 ' flights',
 ' to',
 ' over',
 ' ',
 '150',
 ' destinations',
 ' in',
 ' Europe',
 ',',
 ' Asia',
 ',',
 ' and',
 ' the',
 ' Middle',
 ' East',
 '.',
 ' The',
 ' airline',
 ' is',
 ' headquartered',
 ' at',
 ' Athens',
 ' Ele',
 'f',
 'ther',
 'ios',
 ' Ven',
 'iz',
 'el',
 'os',
 ' International',
 ' Airport',
 ' and',
 ' has',
 ' a',
 ' strong',
 ' focus',
 ' on',
 ' customer',
 ' service',
 ' and',
 ' safety',
 '.\n\n',
 'Would',
 ' you',
 ' like',
 ' to',
 ' know',
 ' something',
 ' specific',
 ' about',
 ' Ae',
 'ge',
 'an',
 ' Airlines',
 ',',
 ' such',
 ' as',
 ' our',
 ' history',
 ',',
 ' fleet',
 ',',
 ' or',
 ' destinations',
 '?',
 '']

## 5. Streaming UI Chatbot

In [56]:
models_available = {
    'Llama 3.1': 'meta-llama/Meta-Llama-3.1-8B-Instruct',
    'Gemma 2': 'google/gemma-2-9b-it'
}

# Load the model and initialize the chat history
history = init_prompts()
tokenizer, model = load_model(models_available["Llama 3.1"])

# Initialize a method for chatting with the LLM using streaming
def gradio_chat_stream(user_input, history):
    if not tokenizer or not model:
        yield "Model not loaded. Please check the configuration.", history, ""
        return

    # Start streaming tokens from the model
    stream = llm_response_stream(tokenizer, model, history, user_input)
        
    # Initialize the chat history for display
    chat_history = [
        (entry["content"], None) if entry["role"] == "user" else (None, entry["content"]) 
        for entry in history if entry["role"] != "system"
    ]

    # Add the user's input to the chat display
    chat_history.append((user_input, None))
    yield chat_history, history, ""  # Show the user's input immediately

    # Stream the assistant's response token by token
    assistant_response = ""
    for token in stream:
        assistant_response += token
        chat_history[-1] = (user_input, assistant_response)  # Update the assistant's response in the last chat entry
        yield chat_history, history, ""  # Update the chatbot dynamically

    # Finalize the assistant's response in the chat history
    chat_history[-1] = (user_input, assistant_response)
    yield chat_history, history, ""  # Ensure the final state is displayed

# Set the Gradio UI
with gr.Blocks() as chat_interface:
    # Title
    gr.Markdown("### Aegean AI Chat Assistant", elem_id="title")

    # Add a dropdown for selecting models
    model_dropdown = gr.Dropdown(
        label="Select Model",
        choices=list(models_available.keys()),
        value=list(models_available.keys())[0],
        interactive=True
    )

    # Chatbox, Input, Send Button
    chatbot = gr.Chatbot()
    user_input = gr.Textbox(label="Your Message", placeholder="Type your message here...", lines=1)
    send_button = gr.Button("Send")

    # Bind send button and text box to chat logic
    send_button.click(
        gradio_chat_stream, 
        inputs=[user_input, gr.State(history)], 
        outputs=[chatbot, gr.State(history), user_input]
    )
    user_input.submit(
        gradio_chat_stream, 
        inputs=[user_input, gr.State(history)], 
        outputs=[chatbot, gr.State(history), user_input]
    )

# Launch the Gradio interface
chat_interface.launch()

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/home/kostasbouros/.local/lib/python3.10/site-packages/gradio/components/chatbot.py:242: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


## 6. RAG

In [2]:
# Define paths
knowledge_base_dir = "RAG/knowledge_base"
vector_store_dir   = "RAG/vector_store"

In [3]:
# Grab all PDF files in the directory
pdf_files = [f for f in os.listdir(knowledge_base_dir) if f.lower().endswith(".pdf")]

# This list will hold exactly 1 Document per PDF
all_docs = []

for pdf_file in pdf_files:
    pdf_path = os.path.join(knowledge_base_dir, pdf_file)
    loader = PyPDFLoader(pdf_path)
    pages = loader.load()  # This typically returns multiple Document objects (one per page)
    
    # Combine text from all pages into one big string
    combined_text = "\n".join(page.page_content for page in pages)
    
    # Create a single Document with all pages combined
    single_doc = Document(
        page_content=combined_text,
        metadata={"source": pdf_file}  # or any other metadata
    )
    all_docs.append(single_doc)

len(all_docs)

39

In [6]:
# Use OpenAI Embeddings 
embeddings = OpenAIEmbeddings()

In [7]:
# Create or load a Chroma vector store using the documents and embeddings
db = Chroma.from_documents(
    documents=all_docs,
    embedding=embeddings,
    persist_directory=vector_store_dir
)

# Persist the database to disk (so you don’t have to re-embed every time)
db.persist()

/tmp/ipykernel_13250/2449357020.py:9: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  db.persist()


In [18]:
# Create a retriever from the vector store
retriever = db.as_retriever(
    search_type="similarity",   # or "mmr", "similarity_score_threshold", etc.
    search_kwargs={"k": 2}     # number of chunks to retrieve
)

In [12]:
query = "Do you know any Daniel Anderson?"
retrieved_docs = retriever.get_relevant_documents(query)

# Print out each retrieved chunk
for i, doc in enumerate(retrieved_docs, start=1):
    print(f"--- Chunk {i} ---")
    print(doc.page_content)
    print()

--- Chunk 1 ---
DANIEL ANDERSON
Java Software Engineer | Backend Development | Data Analytics
 1  234  555 1234 danielanderson@gmailcom danielanderson.com San Jose, California
SUMMARY
With five years of experience in Java development and a zeal for backend innovation, I thrive in high-performance environments. My biggest achievement includes leading a project to develop a scalable microservices architecture, significantly improving operational efficiency. Proficient in Java, Kubernetes, PostgreSQL, and passionate about advancing the functionalities and reliability of tech products.
EXPERIENCE
Senior Java Software Engineer
NexTech Solutions 06/2019   Present  San Jose, California
Implemented a scalable microservices architecture for a financial product, reducing server costs by 30% while handling a 200% increase in traffic.Led the development and optimization of critical financial applications, improving transaction processing speed by 15%.Streamlined continuous integration/continuous d

In [13]:
query = "Do you know any Gabriel Baker?"
retrieved_docs = retriever.get_relevant_documents(query)

# Print out each retrieved chunk
for i, doc in enumerate(retrieved_docs, start=1):
    print(f"--- Chunk {i} ---")
    print(doc.page_content)
    print()

--- Chunk 1 ---
GABRIEL BAKER
Senior QA Engineer | Agile & Scrum Expert
 1  234  555 1234 gabrielbaker@enhancv.com gabrielbaker.com
Denver, Colorado
SUMMARY
With over 5 years of experience as a Senior QA Engineer, my strengths lie in Agile and Scrum methodologies, with a proven track record in optimizing test processes and effective defect documentation leading to high-quality software solutions.
EXPERIENCE
Senior QA Engineer
TechSolutions Inc
06/2020   Present  Denver, Colorado
Led a team of QA specialists in a high-velocity Agile environment, achieving a 25% reduction in regression testing time through optimized test plans.Authored comprehensive test cases and executed 1500  tests for mobile and web applications, achieving 98% coverage and under 0.1% escape defects.Improved defect identification by 30% by implementing a systematic approach to document software defects using Jira.Streamlined communication between developers and QA by initiating weekly sync-ups, resulting in a 20% fast

In [14]:
query = "Do you know any Michelle Sanders?"
retrieved_docs = retriever.get_relevant_documents(query)

# Print out each retrieved chunk
for i, doc in enumerate(retrieved_docs, start=1):
    print(f"--- Chunk {i} ---")
    print(doc.page_content)
    print()

--- Chunk 1 ---
MICHELLE SANDERS
Software Engineer | Full-Stack Developer
michellesanders@gmail.com linkedin.com Houston, Texas
SUMMARY
Dedicated Software Engineer with over two years of experience, skilled in full-stack development and cloud computing. Managed several technical projects, significantly enhancing system performance. Most notable achievement includes winning a university-level hackathon and contributing to a high-profile product during an internship at Hewlett Packard Enterprise.
EXPERIENCE
Software Engineering Intern
D a t e  p e r i o d
Hewlett Packard Enterprise Houston, TX
Worked on product development using a modern tech stack while following Agile methodologies.
Contributed to the development of a key feature in a product, resulting in a 15% increase in user engagement.Improved system performance by 10% through optimization of existing code base.Assisted in the migration of part of the company's infrastructure to a cloud-based solution.
Junior Software Engineer Pre

In [15]:
query = "Tell me any software engineer with more than a year of experience?"
retrieved_docs = retriever.get_relevant_documents(query)

# Print out each retrieved chunk
for i, doc in enumerate(retrieved_docs, start=1):
    print(f"--- Chunk {i} ---")
    print(doc.page_content)
    print()

--- Chunk 1 ---
First LastSoftware Engineer
WORK EXPERIENCE
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ Resume Worded, London, United KingdomEducation technology startup with 50+ employees and $100m+ annual revenueSoftware Engineer 08/2021 – Present
● Created an innovative feature in the company’s ﬂagship product that enabled50+ HNIs to purchase licenses simultaneously, increasing 2021 Q1 earningsby 71%.● Designed new applications by studying users' needs and converting theminto 20+ software programs.● Invented a RESTful API with Flask/Python in addition to an SQLAlchemy ORMlayer, enabling 2800+ developers to access data without exposingpasswords.● Developed applets for Resume Search Engine that provides 13K users withthe information on the search results page.
Polyhire, London, United KingdomNYSE-listed recruitment and employer branding companyTest Engineer 10/2019 – 07/2021
● Promoted cont

In [21]:
query = "I want to know about software engineers that know or has worked with docker!"
retrieved_docs = retriever.get_relevant_documents(query)

# Print out each retrieved chunk
for i, doc in enumerate(retrieved_docs, start=1):
    print(f"--- Chunk {i} ---")
    print(doc.page_content)
    print()

--- Chunk 1 ---
ZARA GREENE
SOFTWARE ENGINEERING MANAGER
CONTACT
zaragreen@email.com
(123) 456-7890
Mountain View, CA
LinkedIn
Github
EDUCATION
M.S.
Computer Science with a
concentration in Software
Theory or Software Systems
Stanford University
2010 - 2012
Stanford, CA
B.S.
Computer Science
Carnegie Mellon University
2006 - 2010
Pittsburgh, PA
SKILLS
Eclipse
Git
Docker
Amazon Web Services (AWS)
Jenkins
React.js
MySQL
Java
CERTIFICATIONS
AWS Certiﬁed Developer from
Amazon Web Services
WORK EXPERIENCE
Software Engineering Manager
Google LLC
May 2019 - April 2023 / Mountain View, CA
Boosted application performance by 23% through the strategic use of
Docker for containerization and deployment.
Enhanced CI/CD pipelines by integrating Jenkins, accelerating release
cycles by 21%.
Optimized data storage and retrieval with MySQL, contributing to a
29% increase in database performance.
Mentored 5 junior engineers on best practices, increasing their
productivity and improving code quality.
Impro